In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [3]:
fogData = xr.open_dataset('../../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
y=fg.fogdata.sum(dim='stations').sel(time=slice('1980-1-1','2018-12-31'))
y=y/25.0

In [4]:
#ERA5 Input

In [5]:
#Surface data
t2mData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_t2m_daily_NovDecJan.nc')
d2mData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_d2m_daily_NovDecJan.nc')
u10Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_u10_daily_NovDecJan.nc')
v10Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_v10_daily_NovDecJan.nc')
# 850 Data
tuv850Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_uvt_daily_NovDecJan.nc')
z850Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_z_daily_NovDecJan.nc')
#700 Data
w700Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_700_w_daily_NovDecJan.nc')
# Read Orography
OroData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
z500Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_500_z_daily_NovDecJan.nc')

In [6]:
t2mData

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 46, longitude: 180, time: 3772)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T09:00:00 ... 2019-12-31T09:00:00
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * latitude   (latitude) float32 90.0 88.0 86.0 84.0 82.0 ... 6.0 4.0 2.0 0.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Wed Feb 03 16:01:55 2021: cdo daymean ERA5NH_t2m_6hourly_No...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...

In [7]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
d2mD = d2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
tuv850D = tuv850Data.sel(latitude=slice(32,24),longitude=slice(74,86))
z850D = z850Data.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = w700Data.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
zD=z500Data.sel(latitude=slice(32,24),longitude=slice(74,86))

In [8]:

w700D.w.values

array([[[-8.68048668e-02, -2.50017643e-02,  2.01451778e-02, ...,
          1.69173956e-01,  5.91557026e-02,  4.51292992e-02],
        [-1.62353516e-02, -2.23717690e-02, -1.22904778e-02, ...,
          7.00855494e-01,  4.51292992e-02,  5.69641590e-02],
        [ 4.36568260e-03, -5.04243374e-02,  6.26621246e-02, ...,
          9.24680233e-02,  7.93182850e-02,  1.45504713e-01],
        [-2.71933079e-02,  1.00638866e-02,  8.54549408e-02, ...,
          9.20295715e-02,  1.92685127e-02,  3.41713428e-02],
        [-6.83953762e-02, -1.22308731e-01,  3.46097946e-02, ...,
          6.70454502e-02,  3.50480080e-02,  2.97882557e-02]],

       [[ 6.00323677e-02, -2.89466381e-02, -4.38494682e-02, ...,
          2.41935015e-01,  1.15260601e-01,  6.87987804e-02],
        [ 6.52921200e-02, -4.64794636e-02,  8.59022141e-04, ...,
          5.93029022e-01, -7.93533325e-02,  6.44154549e-02],
        [-1.53586864e-02, -1.62195683e-01,  6.31005764e-02, ...,
          1.29725218e-01, -4.34112549e-02,  2.41058

In [9]:
v10D.latitude

<xarray.DataArray 'latitude' (latitude: 5)>
array([32., 30., 28., 26., 24.], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 32.0 30.0 28.0 26.0 24.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y

In [10]:
# Calculate wind speed and relative humidity
ws = ((v10D.v10.values**2)+(u10D.u10.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
rh = (e**((17.625*(d2mD.d2m.values-273.15))/(243.04+(d2mD.d2m.values-273.15)))/e**((17.625*(t2mD.t2m.values-273.15))/(243.04+(t2mD.t2m.values-273.15))))
rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rh)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
#Calculate inv
inv=t2mD.t2m.values-tuv850D.t.values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'
#u shear calculation
ushear=(tuv850D.u.values-u10D.u10.values)/(z850D.z.values/9.81) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [11]:
AOData = xr.open_dataset('../../../../Data/AO_EU/AO_Calculated.nc')
AO5D=AOData.AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

In [12]:
EUData = xr.open_dataset('../../../../Data/AO_EU/EU.nc')
EU5D=EUData.EUVal.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


In [13]:
# sst data
sstData=xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/AnomERA5_sst_daily.nc')
sst1=sstData.sst.sel(latitude=slice(5,-5),longitude=slice(-170%360,-120%360)) # nino3.4 region 5S-5N 170W-120W
sstTS=sst1.mean(dim=['latitude','longitude'], skipna=True) 

SST30D=sstTS.rolling(time=30).mean()

SST30DAll=SST30D[((SST30D.time.dt.month>11) | (SST30D.time.dt.month<2)) & 
             (SST30D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#SST

In [14]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [15]:
# Apply Mask 
t2mD.t2m.values=t2mD.t2m.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.w.values=w700D.w.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask
zD.z.values=zD.z.values*mask

In [16]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).t2m
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).w
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear
zTS1=zD.mean(dim=['latitude','longitude'], skipna=True).z

In [17]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)
zTS=zTS1.shift(time=1)



In [18]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
z=zTS[((zTS.time.dt.month>11) | (zTS.time.dt.month<2)) & (zTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [19]:
print('rh')
print(np.corrcoef(rh,y)[0,1])
print('t2m')
print(np.corrcoef(t2m,y)[0,1])
print('ws')
print(np.corrcoef(ws,y)[0,1])
print('w')
print(np.corrcoef(w,y)[0,1])
print('inv')
print(np.corrcoef(inv,y)[0,1])
print('ushear')
print(np.corrcoef(ushear,y)[0,1])
print('z')
print(np.corrcoef(z,y)[0,1])
print('AO5')
print(np.corrcoef(AO5DAll,y)[0,1])
print('EU5')
print(np.corrcoef(EU5DAll,y)[0,1])
print('SST30')
print(np.corrcoef(SST30DAll,y)[0,1])

rh
0.551984190261254
t2m
-0.27475434162179596
ws
-0.36287914975048996
w
-0.04575990405926304
inv
-0.23197758724160944
ushear
-0.14306905339288264
z
0.02148029925141571
AO5
-0.011599709602252865
EU5
-0.011485224923242176
SST30
0.047266557799266536


In [20]:
# input X
#X       = pd.DataFrame([t2m.values,ws.values,rh.values,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
#                        EU5DAll.values,SST30DAll.values],
#                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D', 'nino3.4SSTAnom']).T[:]
#X       = pd.DataFrame([ws.values,rh.values,inv.values,w.values]).T[:]

X       = pd.DataFrame([t2m.values,ws.values,rh.values,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000
mean,288.988799,1.682259,0.635527,3.913068,0.041244,0.166492,-0.009926,-0.000888
std,1.903641,0.378248,0.078827,1.793125,0.066327,0.106554,0.989822,0.573967
min,282.549988,0.662691,0.426100,-2.955961,-0.357248,-0.329140,-3.285089,-1.579142
25%,287.693489,1.428476,0.580451,2.760270,0.010740,0.095389,-0.632978,-0.395233
50%,289.012192,1.640695,0.628189,4.075044,0.049675,0.167706,0.056255,0.025302
75%,290.363098,1.898656,0.689178,5.275727,0.081129,0.235507,0.691648,0.400121
max,295.146027,3.701493,0.901173,8.246736,0.222363,0.485192,2.959325,1.745344


In [21]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000
mean,-151.839842,-4.448426,-8.063922,-2.182713,-0.621966,-1.562835,0.010030,0.001547
std,0.525526,2.644864,12.691179,0.557916,15.083162,9.388814,1.010701,1.742981
min,-153.617364,-11.577665,-41.781619,-4.319956,-91.242177,-45.234558,-3.334220,-4.791183
25%,-152.197430,-6.222984,-16.931197,-2.541397,-7.558790,-7.827945,-0.626165,-1.195972
50%,-151.833384,-4.739059,-9.245451,-2.132315,1.295151,-1.455797,0.077607,0.081078
75%,-151.460447,-2.935291,0.573751,-1.758733,8.448079,4.518340,0.726403,1.219299
max,-150.140054,9.670895,34.704856,-0.834327,40.565820,26.518973,3.041914,5.304376


In [43]:
y

<xarray.DataArray 'fogdata' (time: 2418)>
array([0.32, 0.4 , 0.4 , ..., 0.24, 0.36, 0.36])
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 1980-01-02 ... 2018-12-31

In [44]:
dump(X, 'X.joblib') 
dump(y,'y.joblib')

['y.joblib']

In [45]:
X

,0,1,2,3,4,5,6,7
1980-01-01 09:00:00,-151.803446,-4.064469,-0.709943,-1.704749,-33.468375,14.908108,-1.219917,-0.125832
1980-01-02 09:00:00,-151.844542,-1.727495,9.293493,-1.611327,-38.473908,7.829461,-1.512209,0.515683
1980-01-03 09:00:00,-152.029096,-6.590885,11.797603,-1.290181,-21.173405,-2.099535,-1.533574,0.727990
1980-01-04 09:00:00,-152.365818,-3.041019,9.825014,-1.350949,5.973481,-11.542335,-1.431413,1.267298
1980-01-05 09:00:00,-152.596186,-2.287107,3.897496,-1.649611,-1.177278,-12.862747,-1.304209,1.576376
...,...,...,...,...,...,...,...,...
2018-12-27 09:00:00,-152.356585,-6.189951,-15.593767,-2.078616,-15.912171,11.503087,-0.082453,0.904932
2018-12-28 09:00:00,-152.377714,-2.942925,-22.232640,-1.605875,27.278408,5.756574,-0.028856,1.962011
2018-12-29 09:00:00,-152.615024,-1.262835,-21.464061,-2.027109,21.319440,-3.560168,0.034292,2.274896
2018-12-30 09:00:00,-152.634561,-4.771283,-19.532871,-2.785987,7.737331,-9.083744,0.131938,2.339727
